# Optuna Fashion MNIST CNN Model Usage

In [1]:
import os
import optuna

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms

### 1. Set HyperParameter

In [2]:
BATCHSIZE = 128
CLASSES = 10
EPOCHS = 10
DIR = os.getcwd()

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(DEVICE)

N_TRAIN_EXAMPLES = BATCHSIZE * 469 # Fashion MNIST Full Dataset size
N_VALID_EXAMPLES = BATCHSIZE * 79

cuda


### 2. Get FashionMNIST to dataloader

In [3]:
def get_mnist():
    # Load FashionMNIST dataset.
    train_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST(DIR, train=True, download=True, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )
    valid_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST(DIR, train=False, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )
    return train_loader, valid_loader

### 3. Trial Model

In [4]:
class ConvNets(nn.Module):
    def __init__(self, trial):
        super().__init__()
        self.out_layer = None
        self.layer = self.define_model(trial)
        self.in_features = None
        self.in_channel = None


    def define_model(self, trial):
        n_layers = trial.suggest_int("n_layers", 1, 2)
        layers = []
        self.in_features = 28
        self.in_channel = 1

        for i in range(n_layers):
            out_channel = trial.suggest_int(f"conv_c{i}", 32, 512),
            kernel_size = trial.suggest_int(f"conv_k{i}", 3, 5)
            stride = trial.suggest_int(f"conv_s{i}", 1, 2)
            padding = trial.suggest_int(f"conv_p{i}", 0, 2)
            layers.append(nn.Conv2d(self.in_channel, out_channel[0], kernel_size, stride, padding))
            layers.append(nn.BatchNorm2d(out_channel[0]))
            layers.append(nn.ReLU())
            layers.append(nn.MaxPool2d(kernel_size=2,stride=2))

            out_features = (self.in_features - kernel_size + 2*padding) // (stride) + 1
            out_features = out_features // 2
            print(f"conv_layer_{i}:", self.in_channel, out_channel, self.in_features, out_features, kernel_size, stride, padding)
            self.in_features = out_features
            self.in_channel = out_channel[0]

        print('pow(self.in_features,2)* self.in_channel:',pow(self.in_features,2)* self.in_channel)
        p = trial.suggest_float("dropout_l{}".format(i), 0.0, 0.5)
        layers.append(nn.Flatten())

        layers.append(nn.Linear(pow(self.in_features,2)* self.in_channel, 1000))
        layers.append(nn.Dropout(p))
        layers.append(nn.Linear(1000, 10))
        layers.append(nn.LogSoftmax(dim=1))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.layer(x)

    def forward(self, x):
        x = self.layer(x)
        return x

### 4. objective funtion for optuna

In [5]:
def objective(trial):
    model = ConvNets(trial).to(DEVICE)

    optimizer_name = trial.suggest_categorical("optimizer", ["Adam"])# "RMSprop"])
    lr = trial.suggest_float("lr", 1e-5, 1e-3, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    
    train_loader, valid_loader = get_mnist()
    #epochs = trial.suggest_int("epochs", 1, EPOCHS)

    #with tqdm(range(EPOCHS), total=EPOCHS, unit='epoch') as bar:
    for epoch in range(EPOCHS):
        #bar.set_description(f"Epoch {epoch}")

        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            if batch_idx * BATCHSIZE >= N_TRAIN_EXAMPLES:
                break
            data, target = data.to(DEVICE), target.to(DEVICE)
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()

        model.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(valid_loader):
                if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
                    break
                
                data, target = data.to(DEVICE), target.to(DEVICE)
                output = model(data)

                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / min(len(valid_loader.dataset), N_VALID_EXAMPLES)
        trial.report(accuracy, epoch)

        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy

### 5. Run optuna study

In [7]:
if __name__ == "__main__":
    study = optuna.create_study(directions=["maximize"])
    study.optimize(objective, n_trials=100)

[I 2021-12-03 01:31:04,893] A new study created in memory with name: no-name-e97b3063-bec2-4ab9-bd2f-8cd70ebaad87


conv_layer_0: 1 (437,) 28 13 3 1 0
conv_layer_1: 437 (177,) 13 5 3 1 0
pow(self.in_features,2)* self.in_channel: 4425


[I 2021-12-03 01:32:32,503] Trial 0 finished with value: 0.9021 and parameters: {'n_layers': 2, 'conv_c0': 437, 'conv_k0': 3, 'conv_s0': 1, 'conv_p0': 0, 'conv_c1': 177, 'conv_k1': 3, 'conv_s1': 1, 'conv_p1': 0, 'dropout_l1': 0.03260415135591671, 'optimizer': 'Adam', 'lr': 0.0002935346978904121}. Best is trial 0 with value: 0.9021.


conv_layer_0: 1 (54,) 28 14 3 1 1
pow(self.in_features,2)* self.in_channel: 10584


[I 2021-12-03 01:33:31,466] Trial 1 finished with value: 0.9028 and parameters: {'n_layers': 1, 'conv_c0': 54, 'conv_k0': 3, 'conv_s0': 1, 'conv_p0': 1, 'dropout_l0': 0.2758804391215436, 'optimizer': 'Adam', 'lr': 4.36900407387453e-05}. Best is trial 1 with value: 0.9028.


...

conv_layer_0: 1 (453,) 28 15 3 1 2
conv_layer_1: 453 (396,) 15 7 4 1 1
pow(self.in_features,2)* self.in_channel: 19404


[I 2021-12-03 02:48:34,838] Trial 73 pruned. 


conv_layer_0: 1 (218,) 28 15 3 1 2
conv_layer_1: 218 (352,) 15 7 4 1 1
pow(self.in_features,2)* self.in_channel: 17248


[I 2021-12-03 02:49:00,846] Trial 74 pruned. 


conv_layer_0: 1 (394,) 28 15 3 1 2
conv_layer_1: 394 (305,) 15 7 4 1 1
pow(self.in_features,2)* self.in_channel: 14945
